In [1]:
# Got dataset used in this notebook from: Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.

In [2]:
import torch
import torch.nn as nn

In [3]:
# Get data
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Get training data
x_train = torch.tensor([], dtype=torch.float64, requires_grad=True)
y_train = torch.tensor([], dtype=torch.long)
for i in range(1, 6):
    file = f"../data/CifarTen/data_batch_{i}"
    batch_dict = unpickle(file)
    batch_data = torch.tensor(batch_dict[b"data"].astype(float))
    label_data = torch.tensor(batch_dict[b"labels"])
    x_train = torch.cat((x_train, batch_data))
    y_train = torch.cat((y_train, label_data))

# Get test data
file = f"../data/CifarTen/test_batch"
batch_dict = unpickle(file)
x_test = torch.tensor(batch_dict[b"data"].astype(float), requires_grad=True).type(torch.float64)
y_test = torch.tensor(batch_dict[b"labels"]).type(torch.long)

In [4]:
x_train = x_train.reshape((-1, 3, 32, 32)) # Image size
x_test = x_test.reshape((-1, 3, 32, 32))

In [5]:
# Build Convolutional block
class ConvBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, pool_size, kernel_stride = 1, pool_stride = 2):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=kernel_stride, bias=True, dtype=torch.float64)
        self.batch_norm = nn.BatchNorm2d(out_channels, momentum=0.9, dtype=torch.float64)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_stride)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.act(x)
        x = self.pool(x)
        return x

class MLP(nn.Module):

    def __init__(self, size: tuple):
        super().__init__()
        layers = []
        for i in range(len(size) - 1):
            layers.append(nn.Linear(size[i], size[i+1], bias=True, dtype=torch.float64))
            if i != len(size) - 2:
                layers.append(nn.ReLU())
        self.seq = nn.Sequential(*layers)
    def forward(self, x):
        return self.seq(x)

class Model(nn.Module):

    def __init__(self, blocks):
        super().__init__()
        self.blocks = nn.ModuleList(blocks)
    
    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x
    


In [6]:
# Instantiate model
model = Model([
    ConvBlock(3, 16, 3, 2),
    ConvBlock(16, 32, 3, 2),
    nn.Flatten(),
    MLP((32 * 6 * 6, 128, 10))  # adjust shape if needed
])

# Define loss
criterion = nn.CrossEntropyLoss()

In [7]:
# Hyper params
BATCH_SIZE = 32
STEPS = 5000
LR = 0.1

In [8]:
SEED = 42

for _ in range(STEPS):

    # Minibatch Construct
    ix = torch.randint(0, x_train.shape[0], (BATCH_SIZE,))

    # Forward Pass
    logits = model.forward(x_train[ix])
    loss = nn.functional.cross_entropy(logits, y_train[ix])

    # Backward Pass
    model.zero_grad()
    loss.backward()

    lr = LR if _ < 1000 else LR * 0.1

    # Update
    for p in model.parameters():
        p.data += -lr * p.grad
    
    if _ % 100 == 0:
        print(loss.item())
    

print(loss.item())

2.319170074453515
2.1448509080228684
1.6530848667811995
1.610709650630324
1.5990883817958768
1.4389702006913991
1.6528022278393961
1.5197069542072772
1.0551906905321693
1.1688536495771995
1.186688071202301
0.8677391923220654
1.0521116207616636
1.2998654642620944
0.7352539217462832
1.2261433958779495
1.549701737679205
0.9827741997326148
0.882100326047591
1.0533586230689433
1.0800538553786385
0.7775625240458592
0.9380130525567743
1.182557986287198
0.9423193985559569
0.9144599158450178
0.9633475799389444
0.7101208253420129
0.9561990995669974
0.7284268673618639
1.007280360074493
1.0323159974796883
1.2452140684826238
0.6162432710346059
0.8916131001696659
1.0433787351412862
0.9799732281464218
0.9941164238921777
0.6397303926776704
1.087816236447561
0.9407719525372135
0.7286320557347771
0.7397748206678826
0.46888375224720535
0.7320679003159535
0.9873515372188003
0.9695654210596446
0.912342762561511
0.6412535465788062
0.9466681649575222
0.8210900644781709


In [ ]:
logits = model.forward(x_test[:1000])
print(f"Loss: {nn.functional.cross_entropy(logits, y_train[:1000]).item()}")
print(f"Accuracy: {torch.mean((torch.argmax(logits, axis=1) == y_test[:1000]).type(torch.float64))}")


Loss: 5.199319252421692
Accuracy: 0.649
